In [2]:
# data manipulation

import geopandas as gpd
import pandas as pd
import numpy as np
import networkx as nx
from shapely import wkt
from shapely.geometry import LineString, Point, MultiLineString

# database connection
from sshtunnel import SSHTunnelForwarder
import psycopg2

# plotting libraries
import geoplot.crs as gcrs
import folium
import plotly.express as px



In [3]:
# SSH tunnel configuration
ec2_public_dns = 'ec2-13-238-251-201.ap-southeast-2.compute.amazonaws.com'
ssh_username = 'ubuntu'
ssh_pkey = 'PedalSafe-Vijay_Keypair.pem'
rds_instance_access_point = 'database-ec2-1.cxkqw8uo29xn.ap-southeast-2.rds.amazonaws.com'
rds_port = 5432  # PostgreSQL default port

# Establish SSH tunnel
def make_connection():
    try:
        tunnel = SSHTunnelForwarder(ec2_public_dns,
                                    ssh_username=ssh_username,
                                    ssh_pkey=ssh_pkey,
                                    remote_bind_address=(rds_instance_access_point, rds_port))
        tunnel.start()
        print("****SSH Tunnel Established****")

        connection = psycopg2.connect(
            dbname="PedalSafe_EC2_Master_DB",
            user="postgres",
            password="miBjRVM9LG3GdYS",
            host='127.0.0.1',  # Connect to localhost since we're tunneling
            port=tunnel.local_bind_port
        )
        print('Database connected')
        cursor = connection.cursor()
        return connection, cursor
    except psycopg2.Error as e:
        print("Error establishing connection:", e)
        raise

def exec_query(connection, query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        cursor.close()
        connection.commit()
    except psycopg2.Error as e:
        print("Error executing query:", e)
        raise

In [6]:
# accident_df = pd.read_csv('../data/accident.csv')
# bikelanes_df = pd.read_csv('../data/bikelanes.csv')
# boundary_df = pd.read_csv('../data/boundary.csv')
# all_roads_df = pd.read_csv('../data/all_roads.csv')
# crime_df = pd.read_csv('../data/crimes.csv')

In [20]:
# fetch data from database
conn, cur = make_connection()
#  accident dataframe
cur.execute("SELECT * FROM accident;")
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
accident_df = pd.DataFrame(data=rows, columns=columns)
accidents_gdf = gpd.GeoDataFrame(
    accident_df,
    geometry=gpd.points_from_xy(accident_df.longitude, accident_df.latitude)
)

# boundary dataframe
cur.execute("SELECT * FROM Boundary;")
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
boundary_df = pd.DataFrame(data=rows, columns=columns)
boundary_df['mccid_gis'] = boundary_df['mccid_gis'].astype(int)
boundary_df['polygon_geometry'] = boundary_df['polygon_geometry'].apply(wkt.loads)

# bikelanes dataframe
cur.execute("SELECT * FROM Bikelanes;")
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
bikelanes_df = pd.DataFrame(data=rows, columns=columns)
bikelanes_df['geometry'] = bikelanes_df['geometry'].apply(wkt.loads)
bikelanes_gdf = gpd.GeoDataFrame(bikelanes_df, geometry=bikelanes_df['geometry'])

# all roads dataframe
cur.execute("SELECT * FROM Allroads;")
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
all_roads_df = pd.DataFrame(data=rows, columns=columns)
all_roads_df['geometry'] = all_roads_df['geometry'].apply(wkt.loads)
all_roads_gdf = gpd.GeoDataFrame(all_roads_df, geometry=all_roads_df['geometry'])

# crimes dataset
cur.execute("SELECT * FROM crimes;")
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
crime_df = pd.DataFrame(data=rows, columns=columns)

# bikerails dataset
cur.execute("SELECT * FROM bikerails;")
rows = cur.fetchall()
columns = [desc[0] for desc in cur.description]
bikerails_df = pd.DataFrame(data=rows, columns=columns)
bikerails_df['geometry'] = bikerails_df['geometry'].apply(wkt.loads)
bikerails_gdf = gpd.GeoDataFrame(bikerails_df, geometry=bikerails_df['geometry'])


print('all data fetched ')


****SSH Tunnel Established****
Database connected
all data fetched 


In [5]:
from math import radians, sin, cos, sqrt, atan2
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Earth radius in kilometers

    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # Calculate the differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    # Calculate the distance using the Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c

    return distance

def get_color(accident_count):
    if accident_count == 0:
        return 'green'
    elif 1 <= accident_count < 5:
        return 'yellow'
    elif 5 <= accident_count < 10:
        return 'orange'
    else:
        return 'red'


   
# Extract the cycle lanes and create network graph 
def create_bikelane_network():
    cycle_lanes = []       
    
    G = nx.Graph()
    for idx, row in bikelanes_gdf.iterrows():
        geom = row['geometry']
        accident_count = row['count']
        #accident_count = accident_counts.get(idx, 0)  # Assuming 'accident_counts' maps IDs to counts
        if isinstance(geom, LineString):
            coords = list(geom.coords)
            cycle_lanes.append(geom.coords)
            for i in range(len(coords) - 1):
                G.add_edge(coords[i], coords[i + 1], weight=accident_count)
        elif isinstance(geom, MultiLineString):
            for line in geom.geoms:
                coords = list(line.coords)
                cycle_lanes.append(line.coords)
                for i in range(len(coords) - 1):
                    G.add_edge(coords[i], coords[i + 1], weight=accident_count)


    return cycle_lanes, G   
      
def create_all_roads_network():
    cycle_lanes = []       
    
    G = nx.Graph()
    for idx, row in all_roads_df.iterrows():
        geom = row['geometry']
        accident_count = row['count']
        #accident_count = accident_counts.get(idx, 0)  # Assuming 'accident_counts' maps IDs to counts
        if isinstance(geom, LineString):
            coords = list(geom.coords)
            cycle_lanes.append(geom.coords)
            for i in range(len(coords) - 1):
                G.add_edge(coords[i], coords[i + 1], weight=accident_count)
        elif isinstance(geom, MultiLineString):
            for line in geom.geoms:
                coords = list(line.coords)
                cycle_lanes.append(line.coords)
                for i in range(len(coords) - 1):
                    G.add_edge(coords[i], coords[i + 1], weight=accident_count)


    return cycle_lanes, G           

In [6]:
def blue_route(start_point, end_point, bikelane_network, all_roads_network):
    G = bikelane_network
    G2 = all_roads_network
    destination_text = 'dest_text'
    start_text = 'start_text'
    # Create a map
    accessToken= 'vJATUvPkKU7abTaQMSHw2p9x0jpRE1EcXBTlkPfKH7zeLdtwpN8LyDjoQEXRFjKr'
    tiles = f'https://tile.jawg.io/jawg-terrain/{{z}}/{{x}}/{{y}}{{r}}.png?access-token={accessToken}'
    attr = 	'<a href="https://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank">&copy; <b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',

    map_load_location = [(start_point[1]+end_point[1])/2, (start_point[0]+end_point[0])/2]
    m = folium.Map(location=map_load_location, zoom_start=13,tiles=tiles, attr=attr)  
    distance = 0
    MAX_DISTANCE_THRESHOLD = 1.5

    # Find the nearest nodes in the graph to the start and end points
    start_node_index = np.argmin([Point(node).distance(Point(start_point)) for node in G.nodes])
    end_node_index = np.argmin([Point(node).distance(Point(end_point)) for node in G.nodes])
    start_node = list(G.nodes)[start_node_index]
    end_node = list(G.nodes)[end_node_index]

    

    # Check if the nearest nodes are within the range of the bike lanes
    start_distance = round(haversine(start_point[1], start_point[0], start_node[1], start_node[0]),2)
    end_distance = round(haversine(end_point[1], end_point[0], end_node[1], end_node[0]),2)

    start_far = start_distance > MAX_DISTANCE_THRESHOLD
    end_far = end_distance > MAX_DISTANCE_THRESHOLD
    
    if start_far or end_far:
        message = "Start" if start_far else "End"
        print(f"{message} point is out of range of bike lanes ({start_distance if start_far else end_distance} km away).")
    else:
        try:
            # Find the shortest path between the start and end nodes
            path_nodes = nx.shortest_path(G, source=start_node, target=end_node)
            #path_nodes = nx.astar_path(G, source=start_node, target=end_node, heuristic=lambda n, goal: haversine(n[1], n[0], end_point[1], end_point[0]))

            # Convert the path nodes to coordinates
            path_coords = [[node[1], node[0]] for node in path_nodes]
            total_distance = 0
            for i in range(len(path_coords) - 1):
                total_distance += haversine(path_coords[i+1][0],path_coords[i+1][1], path_coords[i][0],path_coords[i][1])

            # Add the route to the map
            folium.PolyLine(locations=path_coords, color='#1f77b4', weight=10).add_to(m)

            nearest_node = start_node
    
            if start_distance <= MAX_DISTANCE_THRESHOLD:
                
                folium.PolyLine(locations=[(start_point[1], start_point[0]), (nearest_node[1], nearest_node[0])], color='black', weight=2, dash_array='5,5').add_to(m)
            
            if end_distance <= MAX_DISTANCE_THRESHOLD:
                
                folium.PolyLine(locations=[(end_point[1], end_point[0]), (end_node[1], end_node[0])], color='black', weight=5, dash_array='5,5').add_to(m)

        except nx.NetworkXNoPath:
            print("There will be breaks in bicycle lanes")
            start_node_index2 = np.argmin([Point(node).distance(Point(start_point)) for node in G2.nodes])
            end_node_index2 = np.argmin([Point(node).distance(Point(end_point)) for node in G2.nodes])
            start_node2 = list(G2.nodes)[start_node_index2]
            end_node2 = list(G2.nodes)[end_node_index2]
            
            # Find the shortest path between the start and end nodes
            path_nodes2 = nx.shortest_path(G2, source=start_node2, target=end_node2)
            
            # Convert the path nodes to coordinates
            path_coords2 = [[node[1], node[0]] for node in path_nodes2]
            total_distance = 0
            for i in range(len(path_coords2) - 1):
                total_distance += haversine(path_coords2[i+1][0],path_coords2[i+1][1], path_coords2[i][0],path_coords2[i][1])

            # Add the route to the map
            folium.PolyLine(locations=path_coords2, color='#1f77b4', weight=10).add_to(m)
            
            # Add a dotted line between start_point and nearest bike lane node
            # Check if the nearest nodes are within the range of the bike lanes
            start_distance = round(haversine(start_point[1], start_point[0], start_node2[1], start_node2[0]),2)
            end_distance = round(haversine(end_point[1], end_point[0], end_node2[1], end_node2[0]),2)
    
            nearest_node = start_node2
            if start_distance <= MAX_DISTANCE_THRESHOLD:
                
                folium.PolyLine(locations=[(start_point[1], start_point[0]), (nearest_node[1], nearest_node[0])], color='black', weight=5, dash_array='5,5').add_to(m)
            
            if end_distance <= MAX_DISTANCE_THRESHOLD:
                
                folium.PolyLine(locations=[(end_point[1], end_point[0]), (end_node2[1], end_node2[0])], color='black', weight=5, dash_array='5,5').add_to(m)


    folium.CircleMarker(location=(start_point[1],start_point[0]), radius=5, color='#7EA1FF', weight=8,popup=f"{start_text}").add_to(m)
    folium.Marker(location=(end_point[1],end_point[0]),radius=2,color='red', weight=5,popup=f"{destination_text}").add_to(m)        
            
    # display map
    return m, total_distance    
        


In [7]:

_, bikelanes_graph = create_bikelane_network()
_, allroads_graph = create_all_roads_network()
start_tuple = (144.9627652,-37.8102361)
destination_tuple = (144.9703733,-37.8353532) 


# start_tuple = (144.9512, -37.796)
# destination_tuple = (144.98818764532,-37.814201226257) 

fig, route_distance = blue_route(start_tuple, destination_tuple, bikelanes_graph,allroads_graph)
average_speed = 15
average_time = route_distance/average_speed
print(f'Route distance: {round(route_distance,2)} km,\nTime to reach: {round(average_time*60,1)} min' )
fig

There will be breaks in bicycle lanes
Route distance: 3.38 km,
Time to reach: 13.5 min


In [60]:

def colored_route(start_point, end_point, bikelane_network, all_roads_network):
    G = bikelane_network
    G2 = all_roads_network
    destination_text = 'dest_text'
    start_text = 'start_text'
    # Create a map
    accessToken= 'vJATUvPkKU7abTaQMSHw2p9x0jpRE1EcXBTlkPfKH7zeLdtwpN8LyDjoQEXRFjKr'
    tiles = f'https://tile.jawg.io/jawg-terrain/{{z}}/{{x}}/{{y}}{{r}}.png?access-token={accessToken}'
    attr = 	'<a href="https://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank">&copy; <b>Jawg</b>Maps</a> &copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',

    map_load_location = [(start_point[1]+end_point[1])/2, (start_point[0]+end_point[0])/2]
    m = folium.Map(location=map_load_location, zoom_start=13,tiles=tiles, attr=attr)  
    
    MAX_DISTANCE_THRESHOLD = 1.5

    # Find the nearest nodes in the graph to the start and end points
    start_node_index = np.argmin([Point(node).distance(Point(start_point)) for node in G.nodes])
    end_node_index = np.argmin([Point(node).distance(Point(end_point)) for node in G.nodes])
    start_node = list(G.nodes)[start_node_index]
    end_node = list(G.nodes)[end_node_index]
    

    # Check if the nearest nodes are within the range of the bike lanes
    start_distance = round(haversine(start_point[1], start_point[0], start_node[1], start_node[0]),2)
    end_distance = round(haversine(end_point[1], end_point[0], end_node[1], end_node[0]),2)
    
    start_far = start_distance > MAX_DISTANCE_THRESHOLD
    end_far = end_distance > MAX_DISTANCE_THRESHOLD
    
    if start_far or end_far:
        message = "Start" if start_far else "End"
        print(f"{message} point is out of range of bike lanes ({start_distance if start_far else end_distance} km away).")

    else:
        try:
            #Find the shortest path between the start and end nodes
            path_nodes = nx.shortest_path(G, source=start_node, target=end_node)
            path_coords = [[node[1], node[0]] for node in path_nodes]
            total_distance = 0
            for i in range(len(path_coords) - 1):
                total_distance += haversine(path_coords[i+1][0],path_coords[i+1][1], path_coords[i][0],path_coords[i][1])

            for node in path_nodes[:-1]:
                next_node = path_nodes[path_nodes.index(node) + 1]
                color = get_color(G[node][next_node]['weight'])
                # Add the route to the map
                folium.PolyLine(locations=[(node[1], node[0]), (next_node[1], next_node[0])], color=color, weight=10).add_to(m)
            # Add a dotted line between start_point and nearest bike lane node
            nearest_node = start_node
            if start_distance <= MAX_DISTANCE_THRESHOLD:
                
                folium.PolyLine(locations=[(start_point[1], start_point[0]), (nearest_node[1], nearest_node[0])], color='black', weight=5, dash_array='5,5').add_to(m)
            
            if end_distance <= MAX_DISTANCE_THRESHOLD:
                
                folium.PolyLine(locations=[(end_point[1], end_point[0]), (end_node[1], end_node[0])], color='black', weight=5, dash_array='5,5').add_to(m)

                    
        except nx.NetworkXNoPath:
           
            # Display a square area around the start and end points
            print("There will be breaks in bicycle lanes.")
            start_node_index2 = np.argmin([Point(node).distance(Point(start_point)) for node in G2.nodes])
            end_node_index2 = np.argmin([Point(node).distance(Point(end_point)) for node in G2.nodes])
            start_node2 = list(G2.nodes)[start_node_index2]
            end_node2 = list(G2.nodes)[end_node_index2]
            
            # Find the shortest path between the start and end nodes
            path_nodes = nx.shortest_path(G2, source=start_node2, target=end_node2)
            
            path_coords = [[node[1], node[0]] for node in path_nodes]
            total_distance = 0
            for i in range(len(path_coords) - 1):
                total_distance += haversine(path_coords[i+1][0],path_coords[i+1][1], path_coords[i][0],path_coords[i][1])

            # Add the route to the map
            colors = []
            for node in path_nodes[:-1]:
                next_node = path_nodes[path_nodes.index(node) + 1]
                color = get_color(G2[node][next_node]['weight'])
                colors.append(color)
                # Add the route to the map
                folium.PolyLine(locations=[(node[1], node[0]), (next_node[1], next_node[0])], color=color, weight=10).add_to(m)
            
            # Add a dotted line between start_point and nearest bike lane node
            # Check if the nearest nodes are within the range of the bike lanes
            start_distance = round(haversine(start_point[1], start_point[0], start_node2[1], start_node2[0]),2)
            end_distance = round(haversine(end_point[1], end_point[0], end_node2[1], end_node2[0]),2)
    
            nearest_node = start_node2
            if start_distance <= MAX_DISTANCE_THRESHOLD:
                
                folium.PolyLine(locations=[(start_point[1], start_point[0]), (nearest_node[1], nearest_node[0])], color='black', weight=5, dash_array='5,5').add_to(m)
            
            if end_distance <= MAX_DISTANCE_THRESHOLD:
                
                folium.PolyLine(locations=[(end_point[1], end_point[0]), (end_node2[1], end_node2[0])], color='black', weight=5, dash_array='5,5').add_to(m)

            
            
                
    
    folium.CircleMarker(location=(start_point[1],start_point[0]), radius=5, color='#7EA1FF', weight=8,popup=f"{start_text}").add_to(m)
    folium.Marker(location=(end_point[1],end_point[0]),radius=2,color='red', weight=5,popup=f"{destination_text}").add_to(m)        
    
    legend_html = '''
     <div style="position: fixed; 
     bottom: 50px; left: 100px; width: 250px; height: 100px; 
     border:2px solid grey; z-index:9999; font-size:14px;
     background-color: rgba(255, 255, 255, 0.7);
     
     ">&nbsp; <strong> Accident risk</strong>   <br>
     &nbsp; <i class="fa fa-minus" style="color:red"></i> <strong> High Risk </strong>(10+ accidents) <br>
     &nbsp; <i class="fa fa-minus" style="color:orange"></i> <strong>Medium risk </strong> (5-10 accidents)<br>
     &nbsp; <i class="fa fa-minus" style="color:yellow"></i> <strong>Low risk </strong>(1-5 accidents) <br>
     &nbsp; <i class="fa fa-minus" style="color:green"></i> <strong>No accidents </strong> 
      </div>
     '''

    m.get_root().html.add_child(folium.Element(legend_html))


    # display map
    return m, total_distance
        

In [61]:
_, bikelanes_graph = create_bikelane_network()
_, allroads_graph = create_all_roads_network()
start_tuple = (144.9627652,-37.8102361)
destination_tuple = (144.9703733,-37.8353532) 

# start_tuple = (144.9512, -37.796)
# destination_tuple = (144.98818764532,-37.814201226257) 

fig, route_distance = colored_route(start_tuple, destination_tuple, bikelanes_graph,allroads_graph)
average_speed = 15
average_time = route_distance/average_speed
print(f'Route distance: {round(route_distance,2)} km,\nTime to reach: {round(average_time*60,1)} min' )
fig

There will be breaks in bicycle lanes.
Route distance: 3.38 km,
Time to reach: 13.5 min


In [11]:
################################### VISUALISATIONS ###################################################

## Function to create suburb

In [12]:
def find_nearest_suburb(user_lat, user_lon):
    
    min_distance = float('inf')
    nearest_suburb = None
    
    for index, row in boundary_df.iterrows():
        suburb_lat, suburb_lon = eval(row['geo_point_2d'])['lat'], eval(row['geo_point_2d'])['lon']
     
        distance = haversine(user_lat, user_lon, suburb_lat, suburb_lon)
        if distance < min_distance:
            min_distance = distance
            nearest_suburb = row['area_name']
    
    return nearest_suburb

def get_suburb_names(start_coord_tuple, end_coord_tuple ):
    
    start_point = Point(start_coord_tuple)
    end_point = Point(end_coord_tuple)

    nearest_start_suburb = find_nearest_suburb(start_coord_tuple[1], start_coord_tuple[0])
    nearest_end_suburb = find_nearest_suburb(end_coord_tuple[1], end_coord_tuple[0])

    
    start_suburb = boundary_df[boundary_df['polygon_geometry'].apply(lambda x: start_point.within(x))]['area_name'].values
    end_suburb = boundary_df[boundary_df['polygon_geometry'].apply(lambda x: end_point.within(x))]['area_name'].values
    
    start_suburb_name = start_suburb[0] if start_suburb.size > 0 else nearest_start_suburb
    end_suburb_name = end_suburb[0] if end_suburb.size > 0 else nearest_end_suburb
    
    return [start_suburb_name, end_suburb_name]
    

## Function to create bar chart

In [13]:
import plotly.graph_objs as go
def bar_graph(x_array, y_array, marker_color=None):
    if not marker_color:
        marker_color = "DodgerBlue"

    fig = go.Figure(
        data=go.Bar(x=x_array, y=y_array, marker=dict(color=marker_color)),
        layout=go.Layout(
            height=300, margin=dict(l=0, r=0, t=0, b=0))
        )
    
    return fig

def rounded_bar_graph(x_array, y_array, marker_color=None):
    if not marker_color:
        marker_color = "DodgerBlue"

    fig = bar_graph(x_array, y_array)

    bw = 0.4  # half of the bar width
    curve_height = 2.7
    no_color = "rgba(0,0,0,0)"
    shapes = []
    for x, y in zip(fig["data"][0]["x"], fig["data"][0]["y"]):
        path = f"""
            M {x-bw},{y}
            C {x-bw} {y+curve_height}, {x+bw} {y+curve_height}, {x+bw} {y}
            V 0
            M {x+bw},{0}
            C {x+bw} {0-curve_height}, {x-bw} {0-curve_height}, {x-bw} {0}
            H {x-bw}
            Z
        """
        shapes.append(
            dict(type="path", path=path, line_color=no_color, fillcolor=marker_color,)
        )
    fig.update_layout(shapes=shapes,)
    return fig

In [14]:
def plot_accidents_by_time(accident_df, route_suburbs):
    try:
        #'date_time' is a datetime object and extract 'hour'
        accident_df['date_time'] = pd.to_datetime(accident_df['date_time'])
        accident_df['hour'] = accident_df['date_time'].dt.hour

        #Filter the DataFrame for only the suburbs in the route
        filtered_df = accident_df[accident_df['suburb'].isin(route_suburbs)]

        #Check if the dataframe is empty
        if filtered_df.empty:
            raise ValueError("No data available for the specified suburbs.")

        #Group by 'hour' of day and count the total number of accidents
        accident_counts_by_hour = filtered_df.groupby('hour').size().reset_index(name='count')

        bar_fig = rounded_bar_graph(accident_counts_by_hour['hour'],accident_counts_by_hour['count'], marker_color=None)

        #Change x-axis to show labels only once every 3 hours
        tickvals = [i for i in range(0, 24, 3)]  # Every 3rd hour from 0 to 23
        ticktext = [f"{i % 12 if i % 12 != 0 else 12}{'AM' if i < 12 else 'PM'}" for i in tickvals]

        bar_fig.update_xaxes(
            tickvals=tickvals,
            ticktext=ticktext,
            title='Hour of Day'
        )

        #y-axis title
        bar_fig.update_yaxes(title='Accident count')

        #Change the figure size
        bar_fig.update_layout(
            showlegend=False,
            width=400,  # Custom width in pixels
            height=200   # Custom height in pixels
        )

        #Display figure
        return bar_fig

    except KeyError as e:
        print(f"Error: DataFrame missing one or more required columns: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage:
start_tuple = (144.95,-37.825) 
destination_tuple = (144.9717,-37.771) 
route_suburbs = get_suburb_names(start_tuple, destination_tuple)

fig = plot_accidents_by_time(accident_df, route_suburbs)
config = {'displayModeBar': False}
fig.update_layout(
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
)
fig.show(config=config)



## Modified Treemap function

In [15]:
import plotly.graph_objects as go

def create_treemap_top5(crime_df, route_suburbs):
    # Filter crime_df for the suburbs the route passes through
    route_crime_data = crime_df[crime_df['suburb'].isin(route_suburbs)]

    # Aggregate data at the offence subgroup level for the specified suburbs
    route_crime_summary = route_crime_data.groupby('offence_subgroup')['count'].sum().reset_index()

    # Select the top 5 offence subgroups by count
    top5_crime_summary = route_crime_summary.nlargest(5, 'count')

    # Add 'All' as a parent node for hierarchy
    top5_crime_summary['parent'] = 'Top 5 crimes in Suburbs for Selected Route'

    # Create the treemap using Plotly Graph Objects
    fig = go.Figure(go.Treemap(
        labels=top5_crime_summary['offence_subgroup'],
        parents=top5_crime_summary['parent'],
        values=top5_crime_summary['count'],
        marker_colors = ["F08080", "F4978E", "F8AD9D", "FBC4AB", "FFDAB9"],
        texttemplate="%{label}<br>%{value}",
        marker=dict(
            colors=top5_crime_summary['count'],
            #colorscale='RdYlGn_r',
            cmid=0,
            showscale=False
                            )
    ))

    # Customize the layout
    fig.update_layout(
        title='Top 5 Crime Types along the Bicycle Route',
        margin=dict(t=50, l=25, r=25, b=25),
        width =600,
        height=300,
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        
        
        
    )

    # Customize font color
    fig.update_traces(textfont={'color': 'black', 'size': 14})

    return fig

# Example usage
start_tuple = (144.95, -37.825)
destination_tuple = (144.9717, -37.771)
route_suburbs = get_suburb_names(start_tuple, destination_tuple)  # Ensure this function is defined and working
fig = create_treemap_top5(crime_df, route_suburbs)
config = {'displayModeBar': False}
fig.update_traces(root_color="#F6F5F2")
fig.show(config=config)
# fig.write_image("treemap_top5.png", engine="kaleido")


## Bikerails

In [24]:

import branca
from folium.plugins import MarkerCluster


def create_map(dataframe, access_token, zoom_start=12, show_all = True, dest_lat=None, dest_long=None):

    # Calculate the center of the map from the dataframe
    dest_lat = dest_lat
    dest_long = dest_long
    center_latitude = dataframe.geometry.y.mean()
    center_longitude = dataframe.geometry.x.mean()

    # Setup the tile theme with the provided access token
    tiles = f'https://tile.jawg.io/jawg-terrain/{{z}}/{{x}}/{{y}}{{r}}.png?access-token={access_token}'
    attr = ('<a href="https://jawg.io" title="Tiles Courtesy of Jawg Maps" target="_blank">&copy; <b>Jawg</b>Maps</a> &copy; '
            '<a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors')

    # Create the map centered around the average location
    mymap = folium.Map(location=[center_latitude, center_longitude], zoom_start=zoom_start, tiles=tiles, attr=attr)

    # Create a MarkerCluster object
    marker_cluster = MarkerCluster(disableClusteringAtZoom=18).add_to(mymap)
    if show_all == True:
        # Add points to the marker cluster from the dataframe
        for idx, row in dataframe.iterrows():
            html = f"<b>Asset Type</b>: {row['asset_type']} </br> <b>Description</b>: {row['description']}"
            iframe = branca.element.IFrame(html=html, width=400, height=50)
            popup = folium.Popup(iframe, max_width=500)
            
            folium.Marker(
                location=[row.geometry.y, row.geometry.x],
                popup=popup,
                icon=folium.Icon(icon='fa-solid fa-square-parking', prefix='fa'),
            ).add_to(marker_cluster)
    

        return mymap
    else:
        new_df = dataframe.copy()
        new_df['distance'] = new_df.apply(lambda row: haversine(row['geometry'].y, row['geometry'].x, dest_lat, dest_long), axis=1)
    
        # Sort DataFrame based on distance and select the 'n=10' nearest bicycle rails
        nearest_bikerails = new_df.nsmallest(10, 'distance')
        for idx, row in nearest_bikerails.iterrows():
            html = f"<b>Asset Type</b>: {row['asset_type']} </br> <b>Description</b>: {row['description']}"
            iframe = branca.element.IFrame(html=html, width=400, height=50)
            popup = folium.Popup(iframe, max_width=500)
            
            folium.Marker(
                location=[row.geometry.y, row.geometry.x],
                popup=popup,
                icon=folium.Icon(icon='fa-solid fa-square-parking', prefix='fa'),
            ).add_to(mymap)


        return mymap





# Example 
access_token = 'vJATUvPkKU7abTaQMSHw2p9x0jpRE1EcXBTlkPfKH7zeLdtwpN8LyDjoQEXRFjKr'
mymap = create_map(bikerails_gdf, access_token, show_all = False, dest_lat=-37.8353532, dest_long=144.9703733)
mymap
